# Create trees for Cross Validation
## Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/gdrive')
# %cd /gdrive/My Drive/Academic/Data_Science/capstone/colab

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


### Path to reade original tree

In [0]:
% cp /gdrive/My\ Drive/Academic/Data_Science/capstone/newsCapstone/colab/search_tree_with_text.json ./

## Import Packages

In [0]:
import os
import json
import numpy as np 
import pickle as pkl
from sklearn.model_selection import KFold

## Read original tree

In [2]:
import time
start = time.time()
tree = json.load(open('search_tree_with_text.json','r'))
tree = json.loads(tree)
end = time.time()
round((end-start)/60, 2)

0.49

Helper funtion I used to explore the number of texts within each (sub)category

In [20]:
def explore_tree(tree, level=0, lst=[], output=True, key=''):
  if level>3:
    return
  
  append = True
  for k,v in tree.items():
    if k=='pages':
      pass
    elif k=='texts':
      lst.append([2*(level-1)*' '+key, len(tree['texts'])])
    else:
      explore_tree(v, level+1, lst, False, k)

  if output:
    return lst

lst = explore_tree(tree)
lst[:20]

[['Leisure activities', 19],
 ['  Tourist activities', 23],
 ['    Tourist attractions', 12],
 ['    Museums', 2],
 ['    Shopping', 0],
 ['  Bathing', 1],
 ['    Bathing in Australia', 18],
 ['    Hydrotherapy', 21],
 ['    Bathing in Japan', 10],
 ['    Swimming', 45],
 ['    Bathing in art', 47],
 ['    Bathing in Oslo', 2],
 ['    Seabaths', 1],
 ['    Public baths', 12],
 ['    Bathrooms', 36],
 ['    Sauna', 23],
 ['    Ottoman baths', 11],
 ['    Ancient Roman baths', 27],
 ['    Jet ski', 2],
 ['    Bathing in Denmark', 0]]

## Create 5-Fold cross validation trees

In [0]:
trees_train = {1:{}, 2:{}, 3:{}, 4:{}, 5:{}}
trees_val = {1:{}, 2:{}, 3:{}, 4:{}, 5:{}}

def fill_with_None(lst, n=5):
  if len(lst)<n:
    texts = lst.copy() + [None]*n
    return texts
  else:
    return lst.copy()

for key1, values1 in tree.items():
  # print(key1, '(', len(values1), ')')
  tree1 = values1
  texts = fill_with_None(tree1['texts'], 5)

  fold = KFold().split(texts)
  for i in range(1, 6):
    trees_train[i][key1] = {}
    trees_val[i][key1] = {}

    train_idx, val_idx = fold.__next__()
    trees_train[i][key1]['texts'] = [texts[txt] for txt in train_idx if texts[txt] is not None]
    trees_val[i][key1]['texts'] = [texts[txt] for txt in val_idx if texts[txt] is not None]

  for key2, values2 in tree1.items():
    # print('. ', key2)
    if key2 in ['pages','texts']:
      continue
      
    tree2 = values2
    texts = fill_with_None(tree2['texts'], 5)

    fold = KFold().split(texts)
    for j in range(1, 6):
      trees_train[j][key1][key2] = {}
      trees_val[j][key1][key2] = {}

      train_idx, val_idx = fold.__next__()
      trees_train[j][key1][key2]['texts'] = [texts[txt] for txt in train_idx if texts[txt] is not None]
      # display(trees_train[i][key1][key2])
      trees_val[j][key1][key2]['texts'] = [texts[txt] for txt in val_idx if texts[txt] is not None]
    
    for key3, values3 in tree2.items():
      # print('.  . ', key3)
      if key3 in ['pages','texts']:
        continue
        
      tree3 = values3
      texts = fill_with_None(tree3['texts'], 5)

      fold = KFold().split(texts)
      for k in range(1, 6):
        trees_train[k][key1][key2][key3] = {}
        trees_val[k][key1][key2][key3] = {}

        train_idx, val_idx = fold.__next__()
        trees_train[k][key1][key2][key3]['texts'] = [texts[txt] for txt in train_idx if texts[txt] is not None]
        trees_val[k][key1][key2][key3]['texts'] = [texts[txt] for txt in val_idx if texts[txt] is not None]
  
  # break

## Export Trees to pickle files

In [11]:
def pickle_dict(file_name, dct):
  with open(file_name, 'wb') as f:
    pkl.dump(dct, f)

for i in range(1,6):
  file_name = 'tree_train_' + str(i) + '.pkl'
  print(file_name)
  pickle_dict(file_name, trees_train[i])

  file_name = 'tree_val_' + str(i) + '.pkl'
  print(file_name)
  pickle_dict(file_name, trees_val[i])



tree_train_1.pkl
tree_val_1.pkl
tree_train_2.pkl
tree_val_2.pkl
tree_train_3.pkl
tree_val_3.pkl
tree_train_4.pkl
tree_val_4.pkl
tree_train_5.pkl
tree_val_5.pkl


## Copy files back to Google Drive

In [0]:
% cp {tree_train_1.pkl,tree_train_2.pkl,tree_train_3.pkl,tree_train_4.pkl,tree_train_5.pkl,tree_val_1.pkl,tree_val_2.pkl,tree_val_3.pkl,tree_val_4.pkl,tree_val_5.pkl} /gdrive/My\ Drive/Academic/Data_Science/capstone/newsCapstone/colab